In [37]:
import numpy as np
import pandas as pd

from googlesearch import search 
from tqdm import tqdm_notebook as tqdm

import random
from bs4 import BeautifulSoup 
from fuzzywuzzy import fuzz
import warnings
warnings.filterwarnings('ignore')

In [38]:
companies = pd.read_csv('./MarketStatus.csv', skiprows=1)

In [39]:
with open('jobs.json', encoding='UTF-8-SIG') as file:
    data = json.load(file)

In [40]:
comp_list = pd.DataFrame(
    columns=['CompanyName', 'Status', 'Notes', 'JobLink', 'LastUpdated'])

In [41]:
def get_careers_link(company):
    ''' The get careers link gets the name of the company '''
    #     for company in tqdm(companies['Employer']):
    wait_time = 3
    search_string = company + ' + careers'
    this_list = list(search(query_string, tld="com",
                            num=3, stop=3, pause=wait_time))

    # Provided a code option for taking the top three career links from search
#         companies.loc[companies['Employer'] ==
#                       company, 'Careers Link 1'] = this_list[0]
#         companies.loc[companies['Employer'] ==
#                       company, 'Careers Link 2'] = this_list[1]
#         companies.loc[companies['Employer'] ==
#                       company, 'Careers Link 3'] = this_list[2]
    return this_list[0]

In [42]:
def get_status(hiring_stat):
    this_string = None
    if hiring_stat is None:
        return this_string
    
    if len(hiring_stat) > 1:
        j = 1

        for i in hiring_stat:
            if j == 1:
                this_string = data['data']['table']['columns'][1]['typeOptions']['choices'][i]['name']
                j = j + 1
            else:
                this_string = this_string + ', ' + \
                        data['data']['table']['columns'][1]['typeOptions']['choices'][i]['name']
    else:
        try:
            this_string = data['data']['table']['columns'][1]['typeOptions']['choices'][hiring_stat[0]]['name']
        except:
            this_string = None

    return this_string

In [43]:
def get_notes(hiring_notes):

    notes = ''
    try:
        notes = hiring_notes['documentValue'][0]['insert']
    except:
        notes = ''
    return notes

In [44]:
for i in tqdm(data['data']['table']['rows']):
    
    try:
        notes = i['cellValuesByColumnId']['fldb33RYp9aHrTgZU']
    except:
        notes = None
        
    comp_name = i['cellValuesByColumnId']['fldBWaqJq0Z1LZjNH']

    try:
        status = i['cellValuesByColumnId']['fld29mFhwLNN8tJ1i']
    except:
        status = None

    try:
        link = i['cellValuesByColumnId']['fldzqZ8TJUZmyygAL']
    except:
        link = None

    try:
        lastupdate = i['cellValuesByColumnId']['fldOkP0u7XmVKkChC']
    except:
        lastupdate = None

    dictio = [(comp_name, status, notes, link, lastupdate)]

    temp_df = pd.DataFrame.from_records(
        dictio, columns=['CompanyName', 'Status', 'Notes', 'JobLink', 'LastUpdated'])
    comp_list = pd.concat([comp_list, temp_df])

In [45]:
comp_list['Status_Decoded'] = comp_list['Status'].apply(
    lambda x: get_status(x))
comp_list = comp_list.reset_index(drop=True)
comp_list['Notes_Decoded'] = comp_list['Notes'].apply(lambda x: get_notes(x))
comp_list = comp_list.dropna(
    axis='rows', subset=['Status_Decoded']).reset_index(drop=True)
comp_list['CompanyName_lower'] = comp_list['CompanyName'].str.lower()

In [46]:
# comp_list.head()

In [47]:
# companies.head()

In [48]:
# new column for lowercase employer name to be used in matching
companies['Employer_lower'] = companies['Employer'].str.lower()
comp_list['Matched'] = ''

In [49]:
# company list matching with companies master file
# appending information on status and notes
for i in tqdm(comp_list['CompanyName_lower']):
    try:
        test_index = companies.loc[companies['Employer_lower'].str.contains(
            i) == True].index
    except:
        print('Error on matching: ', i)
        test_index = []

    if len(test_index) == 1:
        try:
            companies.loc[companies['Employer_lower'].str.contains(
                i) == True, 'Status'] = comp_list.loc[comp_list['CompanyName_lower'] == i, 'Status_Decoded'].values
            companies.loc[companies['Employer_lower'].str.contains(
                i) == True, 'Notes'] = comp_list.loc[comp_list['CompanyName_lower'] == i, 'Notes_Decoded'].values
        except:
            print('Error on setting', test_index)
    else:
        comp_list.loc[comp_list['CompanyName_lower'] == i, 'Matched'] = False

Error on setting Int64Index([215], dtype='int64')
Error on setting Int64Index([215], dtype='int64')
Error on setting Int64Index([237], dtype='int64')
Error on setting Int64Index([237], dtype='int64')
Error on matching:  belong (belonghome
Error on setting Int64Index([853], dtype='int64')
Error on setting Int64Index([853], dtype='int64')
Error on setting Int64Index([885], dtype='int64')
Error on setting Int64Index([885], dtype='int64')
Error on setting Int64Index([1286], dtype='int64')
Error on setting Int64Index([1286], dtype='int64')
Error on matching:  cwt ((carlsonwagonlit)
Error on setting Int64Index([2028], dtype='int64')
Error on setting Int64Index([2028], dtype='int64')
Error on setting Int64Index([2580], dtype='int64')
Error on setting Int64Index([2580], dtype='int64')
Error on setting Int64Index([2646], dtype='int64')
Error on setting Int64Index([2646], dtype='int64')
Error on setting Int64Index([3002], dtype='int64')
Error on setting Int64Index([3002], dtype='int64')
Error on

In [50]:
# companies.head()
# comp_list.head()

In [51]:
# create new df for unmatched companies
unmatched_comps = comp_list.loc[comp_list['Matched'] == False]

In [52]:
# extract only important columns and rename according to companies masterfile column
temp_unmatched = pd.DataFrame({'Employer': unmatched_comps['CompanyName'],
                               'Job Opportunities Link': unmatched_comps['JobLink'],
                               'Status': unmatched_comps['Status_Decoded'], 
                               'Notes': unmatched_comps['Notes_Decoded']})

In [53]:
# append temp unmatched df to companies
companies = companies.append(temp_unmatched)

In [54]:
# import from coda website source and append to companies masterfile
coda_df = pd.read_csv('./coda-resource.csv',
                      encoding="ISO-8859-1",
                      names=['Employer',
                             'Job Opportunities Link',
                             'Location',
                             'Notes'])
coda_df['Status'] = 'Hiring'
companies = companies.append(coda_df)

In [55]:
# import from startup spreadsheet resource
startup_df = pd.read_csv('./startups.csv', skiprows=7)
startup_df = startup_df[['Company Name', 
                         'HQ Location',
                         'Link to Job Postings', 
                         'Where will roles be located? (post COVID-19)']]
startup_df = startup_df.rename(columns={'Company Name': 'Employer', 
                           'HQ Location': 'HQ State', 
                           'Link to Job Postings': 'Job Opportunities Link', 
                           'Where will roles be located? (post COVID-19)': 'Location'})
startup_df['Status'] = 'Hiring'
companies = companies.append(startup_df)

In [56]:
# drop duplicates on companies and sort in alphabetical order
companies = companies.drop_duplicates(subset=['Employer'], keep='first').reset_index(drop=True)
companies = companies.sort_values('Employer', ascending=True).reset_index(drop=True)
companies['Employer'] = companies['Employer'].str.upper()

In [57]:
# fuzzy matching of company names for those not captured by drop duplicates
for i in tqdm(range(companies.shape[0])):
    ratio_before = 0
    ratio_after = 0
    if i == 0:
        ratio_after = fuzz.ratio(companies['Employer'][i], companies['Employer'][i+1])
    elif i == (companies.shape[0]-1):
        ratio_before = fuzz.ratio(companies['Employer'][i], companies['Employer'][i-1])
    else:
        ratio_after = fuzz.ratio(companies['Employer'][i], companies['Employer'][i+1])
        ratio_before = fuzz.ratio(companies['Employer'][i], companies['Employer'][i-1])
    companies.loc[i, 'Ratio After'] = ratio_after
    companies.loc[i, 'Ratio Before'] = ratio_before

In [58]:
# set thresholds for fuzzy matching
companies.loc[companies['Ratio After'] >= 80, 'Delete'] = 'Review'
companies.loc[companies['Ratio Before'] >= 80, 'Delete'] = 'Review'
companies.loc[companies['Ratio After'] >= 90, 'Delete'] = True

In [ ]:
# get companies without job links
no_link_comps = companies.loc[companies['Job Opportunities Link'].isnull()]['Employer']
# perform a google search for the job link
for company in tqdm(no_link_comps):
    wait_time = random.randint(1,4)
    search_string = company + ' + careers'
#     print(company)
    this_list = list(search(search_string, tld="com", num=1, stop=1, pause=wait_time))
    
    companies.loc[companies['Employer'] == company, 'Job Opportunities Link'] = this_list[0]

In [ ]:
companies.to_csv('./LatestJobMarket.csv', index = False)

## Scratch Area

In [25]:
companies.head()

,Employer,HQ State,Location,ZIP,Post on LinkedIn ?,Job Opportunities Link,Status,Notes / Open Roles,Employer_lower,Notes
0,100 THIEVES,NaN,NaN,NaN,NaN,https://100thieves.com/pages/careers,Hiring,NaN,100 thieves,\n
1,1010DATA,NaN,NaN,NaN,NaN,https://www.1010data.com/company/careers/open-...,Hiring,All roles on website are opened for hiring,1010data,All roles on website are opened for hiring\n
2,10X GENOMICS,NaN,NaN,NaN,NaN,https://www.10xgenomics.com/careers/,"Hiring, Hiring freeze",Actively hiring across the organisation,10x genomics,Actively hiring across the organisation\n
3,11-JUL,NaN,NaN,NaN,NaN,https://sites.7-eleven.com/careers/home,Hiring,NaN,11-jul,NaN
4,12,NaN,NaN,NaN,NaN,https://www.careerplanner.com/ListOfCareers.cfm,Hiring,NaN,12,\n


In [24]:
startup_df[['Company Name', 'HQ Location', 'Link to Job Postings', 'Where will roles be located? (post COVID-19)']]

,Company Name,HQ Location,Link to Job Postings,Where will roles be located? (post COVID-19)
0,TrueAccord,US/California,https://www.trueaccord.com/about-us/careers/,"Remote (post COVID-19), At HQ (as listed above..."
1,PlayVS,US/California,https://www.playvs.com/careers,At HQ (as listed above)
2,Juvo,US/California,https://www.juvo.com/careers,At HQ (as listed above)
3,Hackerone,US/California,https://www.hackerone.com/careers,At HQ (as listed above)
4,Masterclass,US/California,https://careers.masterclass.com/index.html,At HQ (as listed above)
...,...,...,...,...
323,Stel Life Inc.,US/Pennsylvania,NaN,"At HQ (as listed above), Can be remote if nece..."
324,Freedom Financial Network,US/California,https://jobs.freedomfinancialnetwork.com/jobs,At HQ (as listed above)
325,DearDoc,US/New York,https://forms.gle/BJjDa99471smSCzH8,"Remote (post COVID-19), At HQ (as listed above)"
326,Casebook PBC,US/New York,https://www.casebook.net/jobs,"Remote (post COVID-19), At HQ (as listed above)"


In [229]:
for company in tqdm(comp_list.loc[comp_list['Matched'] == False]['CompanyName']):
    wait_time = random.randint(1,4)
    search_string = company + ' + careers'
    this_list = list(search(search_string, tld="com", num=3, stop=3, pause=wait_time))
    
    comp_list.loc[comp_list['CompanyName'] == company, 'Careers Link 1'] = this_list[0]
    comp_list.loc[comp_list['CompanyName'] == company, 'Careers Link 2'] = this_list[1]
    comp_list.loc[comp_list['CompanyName'] == company, 'Careers Link 3'] = this_list[2]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [197]:
companies.shape

(4897, 13)

In [195]:

widgets.Box(
    [
        widgets.Label(value='Pizza topping with a very long label:'),
        widgets.RadioButtons(
            options=[
                'pepperoni',
                'pineapple',
                'anchovies',
                'and the long name that will fit fine and the long name that will fit fine and the long name that will fit fine '
            ],
            layout={'width': 'max-content'}
        )
    ]
)

Box(children=(Label(value='Pizza topping with a very long label:'), RadioButtons(layout=Layout(width='max-cont…

In [ ]:
https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key=YOUR_API_KEY

In [159]:
import time
tic = time.perf_counter()
subset['CompanyName'].apply(lambda x: get_careers_link(x))
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

Downloaded the tutorial in 41.4813 seconds


In [161]:
import time
tic = time.perf_counter()
for query_string in subset['CompanyName']:
    wait_time = 3
    search_string = company + ' + careers'
    this_list = list(search(query_string, tld="com",
                            num=3, stop=3, pause=wait_time))
#     print(this_list[0])
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

Downloaded the tutorial in 39.2813 seconds


In [104]:
from pygeocoder import Geocoder

In [141]:
import requests
from bs4 import BeautifulSoup

company = 'apple'

url = 'http://www.corporate-office-headquarters.com/search-static?term={c}&page=1&rows=200'.format(c=company)

r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

endpoint = soup.find('a', href=True, attrs={'class': "match-result-anchor"})['href']

url = 'http://www.corporate-office-headquarters.com/{ep}'.format(ep=endpoint)

r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

HQ_address = ''
# for i in soup.find_all('dd'):
#     print(i)

print(soup.find_all('dd')[2].text)

# print(HQ_address.text) 

1 Infinite Loop


In [128]:
print(results[0])

1600 Amphitheatre Pkwy, Mountain View, CA 94043, USA


In [125]:
bay_area_city = ['Alameda', 'Albany','American Canyon','Antioch', 'Atherton', 'Belmont', 'Belvedere',
                 'Benicia', 'Berkeley', 'Brentwood',
                'Brisbane',
                'Burlingame','Calistoga', 'Campbell','Clayton',
                'Cloverdale', 'Colma', 'Concord',
                 'Corte Madera', 'Cotati',
                 'Cupertino',
                 'Daly City', 'Danville', 'Dixon',
                 'Dublin',
                 'East Palo Alto',
                 'El Cerrito',
                 'Emeryville',
                 'Fairfax',
                 'Fairfield',
                 'Foster City',
                 'Fremont',
                 'Gilroy',
                 'Half Moon Bay',
                 'Hayward',
                 'Healdsburg',
                 'Hercules',
                 'Hillsborough',
                 'Lafayette',
                 'Larkspur',
                 'Livermore',
                 'Los Altos',
                 'Los Altos Hills',
                 'Los Gatos',
                 'Martinez',
                 'Menlo Park',
                 'Mill Valley',
                 'Millbrae',
                 'Milpitas',
                 'Monte Sereno',
                 'Moraga',
                 'Morgan Hill',
                 'Mountain View',
                 'Napa',
                 'Newark',
                 'Novato',
                 'Oakland',
                 'Oakley',
                 'Orinda', 'Pacifica', 'Palo Alto', 'Petaluma', 'Piedmont',
                 'Pinole', 'Pittsburg', 'Pleasant Hill', 'Pleasanton',
                 'Portola Valley', 'Redwood City', 'Richmond', 'Rio Vista', 'Rohnert Park', 'Ross', 'St. Helena',
                 'San Anselmo', 'San Bruno', 'San Carlos', 'San Francisco', 'San Jose', 'San Leandro', 'San Mateo',
                 'San Pablo', 'San Rafael', 'San Ramon', 'Santa Clara', 'Santa Rosa', 'Saratoga', 'Sausalito',
                 'Sebastopol', 'Sonoma', 'South San Francisco', 'Suisun City', 'Sunnyvale', 'Tiburon', 'Union City',
                 'Vacaville', 'Vallejo', 'Walnut Creek', 'Windsor', 'Woodside', 'Yountville']

In [138]:
new_list = bay_area_companies[['EMPLOYER']]

In [7]:
companies['Employer']

,Unnamed: 0,Fiscal Year,Employer,Initial Approvals,Initial Denials,Continuing Approvals,Continuing Denials,Total Application,NAICS,Tax ID,HQ State,City,ZIP,HQ or Office in N CA,Post on LinkedIn,Unnamed: 15,Unnamed: 16,URL
0,22494,2019,AMAZON.COM SERVICES INC,606,27,124,27,784,45,4687,WA,SEATTLE,98121,1,1,NaN,NaN,NaN
1,22476,2019,GOOGLE LLC,512,16,240,10,778,54,3581,CA,MOUNTAIN VIEW,94043,1,1,NaN,NaN,NaN
2,22472,2019,TECH MAHINDRA AMERICAS INC,475,440,540,78,1533,54,2696,NJ,SOUTH PLAINFIELD,7080,1,1,NaN,NaN,NaN
3,22468,2019,MICROSOFT CORPORATION,364,54,485,15,918,51,4442,WA,REDMOND,98052,1,1,NaN,NaN,NaN
4,22225,2019,LARSEN AND TOUBRO INFOTECH LTD,317,83,417,55,872,54,4303,NJ,EDISON,8817,1,1,NaN,NaN,NaN


In [6]:
import warnings
warnings.filterwarnings('ignore')
import random

In [8]:
for company in tqdm(companies['Employer']):
    wait_time = random.randint(1,4)
    search_string = company + ' + careers'
    this_list = list(search(search_string, tld="com", num=3, stop=3, pause=wait_time))
    
    companies.loc[companies['Employer'] == company, 'Careers Link 1'] = this_list[0]
    companies.loc[companies['Employer'] == company, 'Careers Link 2'] = this_list[1]
    companies.loc[companies['Employer'] == company, 'Careers Link 3'] = this_list[2]

In [224]:
companies.to_csv('./H1B2019.csv', index=False)

In [152]:
new_list.to_csv('./bay_area_companies.csv')

In [158]:
new_list.loc[132,'test']= 'this new value'

In [200]:
companies.shape

(522, 25)

In [231]:
sub_comp_list.loc[sub_comp_list['Careers Link 1'].str.contains('indeed'), 'Careers Link 1'] = ''
sub_comp_list.loc[sub_comp_list['Careers Link 1'].str.contains('glassdoor'), 'Careers Link 1'] = ''
sub_comp_list.loc[sub_comp_list['Careers Link 2'].str.contains('indeed'), 'Careers Link 2'] = ''
sub_comp_list.loc[sub_comp_list['Careers Link 2'].str.contains('glassdoor'), 'Careers Link 2'] = ''
sub_comp_list.loc[sub_comp_list['Careers Link 3'].str.contains('indeed'), 'Careers Link 3'] = ''
sub_comp_list.loc[sub_comp_list['Careers Link 3'].str.contains('glassdoor'), 'Careers Link 3'] = ''

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

In [233]:
sub_comp_list.loc[(sub_comp_list['CompanyName'].str.contains('LINKEDIN') == False) & 
              (sub_comp_list['Careers Link 2'].str.contains('linkedin')), 'Careers Link 2'] = ''
sub_comp_list.loc[(sub_comp_list['CompanyName'].str.contains('LINKEDIN') == False) & 
              (sub_comp_list['Careers Link 3'].str.contains('linkedin')), 'Careers Link 3'] = ''

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [234]:
sub_comp_list.to_csv('./subcomplist.csv', index = False)

In [198]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.7924, 0.2536, 0.2381],
        [0.1356, 0.5010, 0.1841],
        [0.3901, 0.1926, 0.0955],
        [0.9364, 0.4586, 0.9101],
        [0.1144, 0.9713, 0.1289]])


In [25]:
for i in search('stripe analyst open position', tld="com", num=3, stop=3, pause=2):
    print(i)

https://stripe.com/jobs
https://stripe.com/jobs/search
https://stripe.com/jobs/university
